In [1]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm

In [2]:
import tensorflow as tf
import tensorflow_hub as hub

In [3]:
tf.device('cuda:0')

2022-10-20 18:25:49.413347: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-20 18:25:49.558733: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-20 18:25:49.559781: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-20 18:25:49.567381: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [4]:
# train_df=pd.read_excel('../input/ire-major-project-data/train.xlsx')
# dev_df=pd.read_excel('../input/ire-major-project-data/dev.xlsx')
# test_df=pd.read_excel('../input/ire-major-project-data/test.xlsx')

train_df=pd.read_table('../input/ire-major-data-v2/en_train_pt.tsv',names=['SNO','Intensity','Text'],sep='\t')
test_df=pd.read_table('../input/ire-major-data-v2/en_test_pt.tsv',names=['SNO','Intensity','Text'],sep='\t')

In [5]:
from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [6]:
def return_length(df):
    length=[]
    for i in df['Text']:
        tok = tokenizer(i, return_tensors="pt")
        length.append(len(tok['input_ids'][0]))
        
    return length

In [7]:
train_df['#tokens']=return_length(train_df)
test_df['#tokens']=return_length(test_df)

Token indices sequence length is longer than the specified maximum sequence length for this model (573 > 512). Running this sequence through the model will result in indexing errors


In [8]:
train_df=train_df[train_df['#tokens']<50]
test_df=test_df[test_df['#tokens']<50]
print('Length of Train_df:',len(train_df))
print('Length of Test_df:',len(test_df))

Length of Train_df: 17497
Length of Test_df: 4337


In [9]:
train_df['label']=['clickbait' if i>=0.5 else 'no-clickbait' for i in train_df['Intensity']]
test_df['label']=['clickbait' if i>=0.5 else 'no-clickbait' for i in test_df['Intensity']]

## Replacing/Masking Words
- Replace Non Named Entities
- Replace Pronouns
- Replace Random Words

In [10]:
# ! pip install spacy --q
# ! pip install nltk --q
! python -m spacy download en_core_web_sm --q

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [11]:
import spacy
from spacy import displacy
from spacy import tokenizer
nlp = spacy.load('en_core_web_sm')

In [12]:
def get_entities(text):
    doc = nlp(text)
    tokens=[token.text for token in doc]
    ent_tokens=[]
    ent_labels=[]
    for e in doc.ents:
        toks=e.text.split()
        for tok in toks:
            ent_tokens.append(tok)
            ent_labels.append(e.label_)
#     displacy.render(doc, style='ent', jupyter=True)
    #return tokens,ent_tokens,ent_labels
    
    #print(tokens,'\n',ent_tokens,'\n',ent_labels)
    return tokens,ent_tokens,ent_labels

In [13]:
def get_pos_tags(text):
    sent = text
    doc=nlp(sent)

    sub_toks = [tok for tok in doc if ((tok.dep_ == "nsubj") )]
    nc= [x for x in doc.noun_chunks]
    l=[]
    for i,token in enumerate(doc):
        #print(token.pos_)
        if token.pos_ in ['PRON','DET']:
            #l.append([token.text,i,token.pos_])
            l.append(token.text)

    #print(l)
    return l

In [14]:
new_test_sents=[]
for i in test_df['Text']:
    tokens,ent_tokens,ent_labels=get_entities(i)
    pos_tag_tokens=get_pos_tags(i)
    new_tokens=[]
    for token in tokens:
        if ((token not in ent_tokens) & (token in pos_tag_tokens)):
            continue
        else:
            new_tokens.append(token)
    new_text=' '.join(new_tokens).strip()
    new_test_sents.append(new_text)

# Embeddings

In [15]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
model = BertModel.from_pretrained("bert-large-uncased")

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
model.cuda()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 1024, padding_idx=0)
    (position_embeddings): Embedding(512, 1024)
    (token_type_embeddings): Embedding(2, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=Fals

In [17]:
import torch
device=torch.device('cuda:0')

In [18]:
def read_word_embedding(sentence):
    inputs = tokenizer(sentence, return_tensors="pt").to(device)
    outputs = model(**inputs)
    last_hidden_states = outputs.last_hidden_state.cpu()
    embedding=np.mean(last_hidden_states[0].detach().numpy(),axis=0)
#     embedding=np.mean(last_hidden_states[0].detach().to('cpu'),axis=0)
    return embedding

In [19]:
x_train_embeddings=[]
for i in tqdm(range(len(train_df))):
    try:
        x_train_embeddings.append(read_word_embedding(train_df['Text'].iloc[i]))
    except:
        print(i)
        break

100%|██████████| 17497/17497 [06:44<00:00, 43.30it/s]


In [20]:
x_train_embeddings=np.array(x_train_embeddings)
y_train_intensity=np.array(list(train_df['Intensity'].values))

In [21]:
x_test_embeddings=[]
for i in tqdm(range(len(test_df))):
    try:
        x_test_embeddings.append(read_word_embedding(test_df['Text'].iloc[i]))
    except:
        print(i)
        break

100%|██████████| 4337/4337 [01:35<00:00, 45.57it/s]


In [22]:
x_test_embeddings=np.array(x_test_embeddings)
y_test_intensity=np.array(list(test_df['Intensity'].values))
y_test_actual_label=np.array(list(test_df['label'].values))

In [23]:
x_test_embeddings_new=[]
for i in tqdm(new_test_sents):
    try:
        x_test_embeddings_new.append(read_word_embedding(i))
    except:
        print(i)
        break

100%|██████████| 4337/4337 [01:32<00:00, 46.84it/s]


# Linear Regression

In [24]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

In [25]:
linear_reg = LinearRegression().fit(x_train_embeddings, y_train_intensity)

In [26]:
y_test_pred = linear_reg.predict(x_test_embeddings)
y_test_new_pred = linear_reg.predict(x_test_embeddings_new)
y_test_pred_classes=['clickbait' if i>=0.5 else 'no-clickbait' for i in y_test_pred]
y_test_pred_classes_new=['clickbait' if i>=0.5 else 'no-clickbait' for i in y_test_new_pred]
y_test_actual_classes=list(test_df['label'].values)

In [27]:
actual_sents=list(test_df['Text'].values)

In [28]:
out_df=pd.DataFrame(list(zip(actual_sents,y_test_intensity,y_test_pred,new_test_sents,y_test_new_pred)),columns=['actual_sentence','actual_intensity','predicted_intensity','modified_sentence','modified_predicted_intensity'])

In [29]:
# out_df.head()

In [30]:
out_df.to_csv('./Test_Results_LR.csv')

In [31]:
from sklearn.metrics import median_absolute_error,mean_squared_error, f1_score, accuracy_score

In [32]:
def results(y_true,y_pred,y_actual_classes,y_pred_classes):
    Medae = median_absolute_error(y_true,y_pred)
    mse = mean_squared_error(y_true,y_pred) #default=True
    rmse = mean_squared_error(y_true,y_pred,squared=False)
    accuracy=accuracy_score(y_actual_classes,y_pred_classes)
    f1=f1_score(y_actual_classes,y_pred_classes,pos_label='clickbait')
    
    print("MedAE:",Medae)
    print("MSE:",mse)
    print("RMSE:",rmse)
    print("Accuracy:",accuracy)
    print("f1-score:",f1)

In [33]:
print('Without Masking/Replacing Words\n')
results(y_test_intensity,y_test_pred,y_test_actual_classes,y_test_pred_classes)

Without Masking/Replacing Words

MedAE: 0.10960439042890624
MSE: 0.029353670269196528
RMSE: 0.1713291284901564
Accuracy: 0.8445930366612866
f1-score: 0.6463798530954878


In [34]:
print('With Masking/Replacing Words\n')
results(y_test_intensity,y_test_new_pred,y_test_actual_classes,y_test_pred_classes_new)

With Masking/Replacing Words

MedAE: 0.113645426445625
MSE: 0.03176614000242992
RMSE: 0.17823058099672437
Accuracy: 0.8388286834217201
f1-score: 0.5980448533640024


# Ridge Regression

In [35]:
from sklearn.linear_model import Ridge
clf = Ridge(alpha=2).fit(x_train_embeddings, y_train_intensity)

In [36]:
y_test_pred = clf.predict(x_test_embeddings)
y_test_new_pred = clf.predict(x_test_embeddings_new)
y_test_pred_classes=['clickbait' if i>=0.5 else 'no-clickbait' for i in y_test_pred]
y_test_pred_classes_new=['clickbait' if i>=0.5 else 'no-clickbait' for i in y_test_new_pred]
y_test_actual_classes=list(test_df['label'].values)

In [37]:
print('Without Masking/Replacing Words\n')
results(y_test_intensity,y_test_pred,y_test_actual_classes,y_test_pred_classes)

Without Masking/Replacing Words

MedAE: 0.10892980100724375
MSE: 0.0291963610311915
RMSE: 0.17086942684749518
Accuracy: 0.8445930366612866
f1-score: 0.6445147679324895


In [38]:
print('With Masking/Replacing Words\n')
results(y_test_intensity,y_test_new_pred,y_test_actual_classes,y_test_pred_classes_new)

With Masking/Replacing Words

MedAE: 0.11327401395612424
MSE: 0.03161434876767562
RMSE: 0.1778042428281047
Accuracy: 0.8388286834217201
f1-score: 0.5980448533640024


# Gradient Boosted Regression

In [39]:
from sklearn.ensemble import GradientBoostingRegressor
clf = GradientBoostingRegressor().fit(x_train_embeddings, y_train_intensity)

In [40]:
y_test_pred = clf.predict(x_test_embeddings)
y_test_new_pred = clf.predict(x_test_embeddings_new)
y_test_pred_classes=['clickbait' if i>=0.5 else 'no-clickbait' for i in y_test_pred]
y_test_pred_classes_new=['clickbait' if i>=0.5 else 'no-clickbait' for i in y_test_new_pred]
y_test_actual_classes=list(test_df['label'].values)

In [41]:
print('Without Masking/Replacing Words\n')
results(y_test_intensity,y_test_pred,y_test_actual_classes,y_test_pred_classes)

Without Masking/Replacing Words

MedAE: 0.11959921905770182
MSE: 0.031571694975067593
RMSE: 0.17768425640744762
Accuracy: 0.8383675351625548
f1-score: 0.6124930901050303


In [42]:
print('With Masking/Replacing Words\n')
results(y_test_intensity,y_test_new_pred,y_test_actual_classes,y_test_pred_classes_new)

With Masking/Replacing Words

MedAE: 0.12138454951488004
MSE: 0.03448890286593769
RMSE: 0.18571188132679528
Accuracy: 0.8201521789255246
f1-score: 0.5249695493300853


# Random Forest Regression

In [43]:
from sklearn.ensemble import RandomForestRegressor
clf= RandomForestRegressor(max_depth=3, random_state=0).fit(x_train_embeddings, y_train_intensity)

In [44]:
y_test_pred = clf.predict(x_test_embeddings)
y_test_new_pred = clf.predict(x_test_embeddings_new)
y_test_pred_classes=['clickbait' if i>=0.5 else 'no-clickbait' for i in y_test_pred]
y_test_pred_classes_new=['clickbait' if i>=0.5 else 'no-clickbait' for i in y_test_new_pred]
y_test_actual_classes=list(test_df['label'].values)

In [45]:
print('Without Masking/Replacing Words\n')
results(y_test_intensity,y_test_pred,y_test_actual_classes,y_test_pred_classes)

Without Masking/Replacing Words

MedAE: 0.15414561867907123
MSE: 0.0462313179783515
RMSE: 0.21501469247089022
Accuracy: 0.778418261471063
f1-score: 0.32180663373323926


In [46]:
print('With Masking/Replacing Words\n')
results(y_test_intensity,y_test_new_pred,y_test_actual_classes,y_test_pred_classes_new)

With Masking/Replacing Words

MedAE: 0.1577426957878486
MSE: 0.04850369970163641
RMSE: 0.22023555503514053
Accuracy: 0.7632003689186073
f1-score: 0.19702892885066456


# Adaboost Regression

In [47]:
from sklearn.ensemble import AdaBoostRegressor
clf = AdaBoostRegressor(random_state=0, n_estimators=100).fit(x_train_embeddings, y_train_intensity)

In [48]:
y_test_pred = clf.predict(x_test_embeddings)
y_test_new_pred = clf.predict(x_test_embeddings_new)
y_test_pred_classes=['clickbait' if i>=0.5 else 'no-clickbait' for i in y_test_pred]
y_test_pred_classes_new=['clickbait' if i>=0.5 else 'no-clickbait' for i in y_test_new_pred]
y_test_actual_classes=list(test_df['label'].values)

In [49]:
print('Without Masking/Replacing Words\n')
results(y_test_intensity,y_test_pred,y_test_actual_classes,y_test_pred_classes)

Without Masking/Replacing Words

MedAE: 0.16596281076847968
MSE: 0.04324632096729674
RMSE: 0.20795749798287327
Accuracy: 0.8270694028130043
f1-score: 0.5614035087719298


In [50]:
print('With Masking/Replacing Words\n')
results(y_test_intensity,y_test_new_pred,y_test_actual_classes,y_test_pred_classes_new)

With Masking/Replacing Words

MedAE: 0.16862400867256028
MSE: 0.04460749331732833
RMSE: 0.21120486101727945
Accuracy: 0.8072400276688956
f1-score: 0.4441489361702128
